<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1.  <a href="#item1">Download and Explore Dataset</a>

2.  <a href="#item2">Explore Neighborhoods in Toronto</a>

3.  <a href="#item3">Analyze Each Neighborhood</a>

4.  <a href="#item4">Cluster Neighborhoods</a>

5.  <a href="#item5">Examine Clusters</a>  
    </font>
    </div>


Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import ArcGIS # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import time



**Dear reviewer!**

Due to a well known looping issue of the Foursqare sign up procedure (see [Forum](https://www.coursera.org/learn/applied-data-science-capstone/discussions/weeks/2/threads/GBEHazsAEeuipg7jhk5jrQ), I wasn't able to create a developer account to use the API in this exercise.
To proceed with the course, I obtained a developer account on [Yelp](https://www.yelp.de/developers?country=US) and used the Yelp [Yelp Fusion API](https://www.yelp.de/fusion) instead.
To check the code below, please insert your own Yelp API Key or simply import the Yelp data from Excel-file below. 

In [2]:
from yelpapi import YelpAPI
import argparse

MY_API_KEY = "" #  Replace this with your real API key

yelp_api = YelpAPI(MY_API_KEY)


## 1. Download and Explore Dataset

Copied raw data of Toronto boroughs and neighborhoods from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M to csv-file: Toronto.csv

In [3]:
toronto_data = pd.read_csv("Toronto.csv", sep=';')
toronto_data

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


We only need entries with a borough assigned to the postalcode. We also drop PO Boxes from the dataset.

In [4]:
toronto_data = toronto_data[toronto_data['Borough'] != 'Not assigned']
toronto_data = toronto_data[~toronto_data['PostalCode'].isin(['M5W', 'M7Y'])]

We reduce the dataset to "central" Toronto boroughs.

In [5]:
toronto_data = toronto_data[toronto_data['Borough'].str.contains('Toronto')].copy()

In [6]:
toronto_data

,PostalCode,Borough,Neighborhood
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
13,M5B,Downtown Toronto,"Garden District, Ryerson"
22,M5C,Downtown Toronto,St. James Town
30,M4E,East Toronto,The Beaches
31,M5E,Downtown Toronto,Berczy Park
40,M5G,Downtown Toronto,Central Bay Street
41,M6G,Downtown Toronto,Christie
49,M5H,Downtown Toronto,"Richmond, Adelaide, King"
50,M6H,West Toronto,"Dufferin, Dovercourt Village"


Define a function to get the latitude and longitude of the neighborhoods

In [7]:
def getLocationFromPostcode(postcode, country):
    geolocator = ArcGIS(user_agent="toronto_explorer")
    location = geolocator.geocode(query=postcode+' '+country)
    latitude = location.latitude
    longitude = location.longitude
    
    return(latitude, longitude)


In [8]:
toronto_data['Latitude'] = toronto_data['PostalCode'].apply(
    lambda x: getLocationFromPostcode(x, 'Canada')[0])
toronto_data['Longitude'] = toronto_data['PostalCode'].apply(
    lambda x: getLocationFromPostcode(x, 'Canada')[1])
toronto_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804
22,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587
30,M4E,East Toronto,The Beaches,43.67709,-79.29547
31,M5E,Downtown Toronto,Berczy Park,43.64536,-79.37306
40,M5G,Downtown Toronto,Central Bay Street,43.65609,-79.38493
41,M6G,Downtown Toronto,Christie,43.66869,-79.42071
49,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.64970,-79.38258
50,M6H,West Toronto,"Dufferin, Dovercourt Village",43.66505,-79.43891


Get geolocation of Toronto and visualize neighborhoods in map.

In [9]:
# get geolocation to create a map of Toronto
geolocator = ArcGIS(user_agent="toronto_explorer")
toronto_location = geolocator.geocode(query='Toronto')
toronto_lat = toronto_location.latitude
toronto_lng = toronto_location.longitude

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[toronto_lat, toronto_lng], zoom_start=11)

# add neighborhood markers to map
for lat, lng, label in zip(toronto_data['Latitude'], 
                           toronto_data['Longitude'], 
                           toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## 2. Explore Neighborhoods in Toronto

#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [12]:
def getNearbyVenues(neighborhoods, latitudes, longitudes, radius=500):
    
    all_venues = pd.DataFrame(columns = ['Neighborhood',
    'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue',
    'Venue Latitude', 'Venue Longitude', 'Venue Category'])
    
    for neighborhood, lat, lng in zip(neighborhoods, latitudes, longitudes):
        print(neighborhood)    
        # get results from Yelp
        results = yelp_api.search_query(longitude=lng, latitude=lat, radius=radius, limit=50)
        
        #check if there are businesses listed in the neighborhood
        total_result = results['total']
        print(neighborhood, total_result, 'venues')
        
        if total_result > 0:
            
            results_cols = list(results['businesses'][0].keys())
               
            if total_result > 1000:
                total_result = 999
            
            # loop through Yelp results pages by 50 items to get all entries from Yelp
            all_items = []

            for offset in range(0, total_result, 50):
                results = yelp_api.search_query(
                    longitude=lng,
                    latitude=lat,
                    radius=radius,
                    limit=50,
                    offset=offset) 
                all_items.extend(results['businesses'])
                # wait  to slow down api calls
                time.sleep(1)
            
            # initialise temporary dataframe with column names from business dictionary
            data = pd.DataFrame(all_items, columns=results_cols)
            
            # extract categories alias from business dataframe 
            data_categories = data.categories.apply(pd.Series)[0].apply(pd.Series)
            
            # extract coordinates (lat & lng) business dataframe
            data_coordinates = data.coordinates.apply(pd.Series)
            
            # extract venue names from business dataframe
            data_names = data.name

            # combine extracted information to a temporary dataframe to add to all venues dataframe
            add_venues  = pd.concat([data_names, data_categories['alias'], data_coordinates], axis=1)
                
            add_venues['Neighborhood'] = neighborhood
            add_venues['Neighborhood Latitude'] = lat 
            add_venues['Neighborhood Longitude'] = lng 
            add_venues.rename(columns={"name": "Venue", 
                                      "latitude": "Venue Latitude", 
                                      "longitude": "Venue Longitude" , 
                                      "alias": "Venue Category"}, 
                             inplace=True)
             
            all_venues = all_venues.append(add_venues, ignore_index=True)
        
    nearby_venues = all_venues

    return(nearby_venues)

Get all venues for all neighborhoods.

In [13]:
toronto_venues = getNearbyVenues(neighborhoods=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Regent Park, Harbourfront
Regent Park, Harbourfront 61 venues
Queen's Park, Ontario Provincial Government
Queen's Park, Ontario Provincial Government 94 venues
Garden District, Ryerson
Garden District, Ryerson 367 venues
St. James Town
St. James Town 406 venues
The Beaches
The Beaches 8 venues
Berczy Park
Berczy Park 201 venues
Central Bay Street
Central Bay Street 453 venues
Christie
Christie 34 venues
Richmond, Adelaide, King
Richmond, Adelaide, King 541 venues
Dufferin, Dovercourt Village
Dufferin, Dovercourt Village 13 venues
Harbourfront East, Union Station, Toronto Islands
Harbourfront East, Union Station, Toronto Islands 258 venues
Little Portugal, Trinity
Little Portugal, Trinity 120 venues
The Danforth West, Riverdale
The Danforth West, Riverdale 5 venues
Toronto Dominion Centre, Design Exchange
Toronto Dominion Centre, Design Exchange 494 venues
Brockton, Parkdale Village, Exhibition Place
Brockton, Parkdale Village, Exhibition Place 92 venues
India Bazaar, The Beaches West
I

In [14]:
toronto_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65512,-79.36264,Roselle,43.653591,-79.362106,bakeries
1,"Regent Park, Harbourfront",43.65512,-79.36264,Tandem Coffee,43.653720,-79.361900,coffee
2,"Regent Park, Harbourfront",43.65512,-79.36264,Mengrai Thai,43.654100,-79.366917,thai
3,"Regent Park, Harbourfront",43.65512,-79.36264,Mangia & Bevi,43.652224,-79.366307,italian
4,"Regent Park, Harbourfront",43.65512,-79.36264,Rooster Coffee House,43.651788,-79.365487,coffee
5,"Regent Park, Harbourfront",43.65512,-79.36264,Soma Chocolatemaker,43.650661,-79.358266,chocolate
6,"Regent Park, Harbourfront",43.65512,-79.36264,Figs Breakfast & Lunch,43.655815,-79.364713,breakfast_brunch
7,"Regent Park, Harbourfront",43.65512,-79.36264,King Solomon and Queen of Sheba,43.655806,-79.363937,ethiopian
8,"Regent Park, Harbourfront",43.65512,-79.36264,Betty's,43.651577,-79.367438,cocktailbars
9,"Regent Park, Harbourfront",43.65512,-79.36264,Impact Kitchen,43.656063,-79.357131,coffee


Export the venues so that we can reuse them later without calling the Yelp-API again.

In [15]:
toronto_venues.to_excel('toronto_venues.xls', index=False)

Let's check how many venues were returned for each neighborhood

In [16]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,201,201,201,201,201,201
"Brockton, Parkdale Village, Exhibition Place",92,92,92,92,92,92
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",133,133,133,133,133,133
Central Bay Street,453,453,453,443,443,453
Christie,34,34,34,34,34,34
Church and Wellesley,239,239,239,239,239,239
"Commerce Court, Victoria Hotel",487,487,487,485,485,485
Davisville,39,39,39,39,39,39
Davisville North,10,10,10,10,10,10


#### Let's find out how many unique categories can be curated from all the returned venues

In [17]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 175 uniques categories.


## 3. Analyze Each Neighborhood

In [18]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,active,afghani,african,amusementparks,aquariums,arcades,argentine,artmuseums,asianfusion,australian,bagels,bakeries,bars,bbq,beaches,beerbar,bistros,bookstores,brasseries,brazilian,breakfast_brunch,breweries,brewpubs,british,bubbletea,buffets,burgers,cafes,cajun,caribbean,catering,champagne_bars,cheese,cheesesteaks,chicken_wings,chickenshop,chinese,chocolate,cocktailbars,coffee,coffeeroasteries,comedyclubs,comfortfood,convenience,creperies,cuban,customcakes,danceclubs,delicatessen,delis,desserts,dimsum,diners,dinnertheater,divebars,dog_parks,donuts,dumplings,egyptian,ethiopian,falafel,farmersmarket,filipino,fishnchips,food_court,foodstands,foodtrucks,french,galleries,gardens,gastropubs,gaybars,german,gluten_free,gourmet,greek,grocery,hakka,halal,hawaiian,headshops,hiking,himalayan,hkcafe,hookah_bars,hotdog,hotdogs,hotels,hotpot,icecream,indpak,international,irish,irish_pubs,italian,japanese,juicebars,karaoke,kebab,korean,landmarks,laotian,latin,laundromat,lebanese,localflavor,lounges,macarons,malaysian,meats,mediterranean,mexican,mideastern,modern_european,mongolian,moroccan,mountainbiking,museums,musicvenues,newcanadian,noodles,organic_stores,pakistani,panasian,parks,persian,peruvian,pizza,poke,polish,poolhalls,portuguese,postoffices,publicart,pubs,ramen,restaurants,salad,salvadoran,sandwiches,seafood,shavedsnow,singaporean,social_clubs,soup,southern,spanish,speakeasies,sportgoods,sportsbars,steak,streetvendors,sushi,taiwanese,tapas,tapasmallplates,tea,tex-mex,thai,theater,tours,tradamerican,turkish,vapeshops,vegan,vegetarian,venezuelan,venues,vietnamese,waffles,whiskeybars,wine_bars,yelpevents,zoos
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [19]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,active,afghani,african,amusementparks,aquariums,arcades,argentine,artmuseums,asianfusion,australian,bagels,bakeries,bars,bbq,beaches,beerbar,bistros,bookstores,brasseries,brazilian,breakfast_brunch,breweries,brewpubs,british,bubbletea,buffets,burgers,cafes,cajun,caribbean,catering,champagne_bars,cheese,cheesesteaks,chicken_wings,chickenshop,chinese,chocolate,cocktailbars,coffee,coffeeroasteries,comedyclubs,comfortfood,convenience,creperies,cuban,customcakes,danceclubs,delicatessen,delis,desserts,dimsum,diners,dinnertheater,divebars,dog_parks,donuts,dumplings,egyptian,ethiopian,falafel,farmersmarket,filipino,fishnchips,food_court,foodstands,foodtrucks,french,galleries,gardens,gastropubs,gaybars,german,gluten_free,gourmet,greek,grocery,hakka,halal,hawaiian,headshops,hiking,himalayan,hkcafe,hookah_bars,hotdog,hotdogs,hotels,hotpot,icecream,indpak,international,irish,irish_pubs,italian,japanese,juicebars,karaoke,kebab,korean,landmarks,laotian,latin,laundromat,lebanese,localflavor,lounges,macarons,malaysian,meats,mediterranean,mexican,mideastern,modern_european,mongolian,moroccan,mountainbiking,museums,musicvenues,newcanadian,noodles,organic_stores,pakistani,panasian,parks,persian,peruvian,pizza,poke,polish,poolhalls,portuguese,postoffices,publicart,pubs,ramen,restaurants,salad,salvadoran,sandwiches,seafood,shavedsnow,singaporean,social_clubs,soup,southern,spanish,speakeasies,sportgoods,sportsbars,steak,streetvendors,sushi,taiwanese,tapas,tapasmallplates,tea,tex-mex,thai,theater,tours,tradamerican,turkish,vapeshops,vegan,vegetarian,venezuelan,venues,vietnamese,waffles,whiskeybars,wine_bars,yelpevents,zoos
0,Berczy Park,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.004975,0.000000,0.000000,0.004975,0.034826,0.014925,0.009950,0.004975,0.000000,0.004975,0.000000,0.000000,0.000000,0.034826,0.004975,0.00,0.000000,0.000000,0.000000,0.019900,0.029851,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014925,0.000000,0.024876,0.104478,0.004975,0.000000,0.000000,0.000000,0.004975,0.000000,0.000000,0.000000,0.000000,0.024876,0.019900,0.000000,0.000000,0.000000,0.000000,0.000000,0.004975,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.009950,0.004975,0.000000,0.009950,0.004975,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014925,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024876,0.00000,0.000000,0.009950,0.019900,0.000000,0.004975,0.000000,0.044776,0.029851,0.009950,0.000000,0.000000,0.009950,0.019900,0.000000,0.000000,0.000000,0.004975,0.004975,0.019900,0.000000,0.000000,0.000000,0.009950,0.009950,0.014925,0.004975,0.000000,0.004975,0.000000,0.009950,0.000000,0.039801,0.000000,0.000000,0.000000,0.000000,0.014925,0.000000,0.000000,0.034826,0.004975,0.00,0.000000,0.000000,0.000000,0.000,0.039801,0.000000,0.000000,0.009950,0.000000,0.034826,0.019900,0.000000,0.000000,0.000000,0.004975,0.00000,0.000000,0.000000,0.00,0.009950,0.014925,0.000000,0.024876,0.000000,0.004975,0.000000,0.004975,0.000000,0.019900,0.000000,0.000000,0.019900,0.000000,0.000000,0.014925,0.004975,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.065217,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.010870,0.000000,0.032609,0.000000,0.00,0.000000,0.000000,0.000000,0.065217,0.010870,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010870,0.010870,0.000000,0.021739,0.065217,0.000000,0.000000,0.000000,0.010870,0.000000,0.000000,0.010870,0.000000,0.000000,0.000000,0.000000,0.000000,0.010870,0.000000,0.010870,0.010870,0.000000,0.000000,0.000000,0.010870,0.010870,0.000000,0.000000,0.000000,0.000000,0.000000,0.010870,0.000000,0.000000,0.000000,0.000000,0.000000,0.010870,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.

In [20]:
toronto_grouped.shape

(34, 175)

#### Let's print each neighborhood along with the top 5 most common venues


In [21]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
         venue  freq
0       coffee  0.10
1      italian  0.04
2         pubs  0.04
3  newcanadian  0.04
4        pizza  0.03


----Brockton, Parkdale Village, Exhibition Place----
      venue  freq
0     pizza  0.10
1   burgers  0.07
2  bakeries  0.07
3    coffee  0.07
4      bars  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
          venue  freq
0        coffee  0.11
1          bars  0.08
2         pizza  0.06
3  cocktailbars  0.05
4    sandwiches  0.04


----Central Bay Street----
        venue  freq
0      coffee  0.14
1     hotdogs  0.05
2    japanese  0.04
3  sandwiches  0.04
4     chinese  0.03


----Christie----
       venue  freq
0     korean  0.12
1     coffee  0.12
2      cafes  0.09
3  bubbletea  0.06
4    mexican  0.06


----Church and Wellesley----
       venue  freq
0     coffee  0.10
1   japanese  0.06
2    hotdogs  0.04
3      pizza  0.04
4  bubbletea  0.04


----Commerce 

#### Let's put that into a _pandas_ dataframe


First, let's write a function to sort the venues in descending order.


In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.


In [23]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,coffee,italian,newcanadian,pubs,bakeries,sandwiches,breakfast_brunch,pizza,cafes,japanese
1,"Brockton, Parkdale Village, Exhibition Place",pizza,coffee,bakeries,burgers,bars,newcanadian,wine_bars,indpak,breakfast_brunch,vietnamese
2,"CN Tower, King and Spadina, Railway Lands, Har...",coffee,bars,pizza,cocktailbars,sandwiches,italian,breakfast_brunch,mexican,lounges,japanese
3,Central Bay Street,coffee,hotdogs,japanese,sandwiches,landmarks,chinese,thai,desserts,newcanadian,bubbletea
4,Christie,korean,coffee,cafes,bubbletea,parks,mexican,bakeries,salvadoran,landmarks,catering


## 4. Cluster Neighborhoods


Run _k_-means to cluster the neighborhood into 5 clusters.


In [24]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.


In [27]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

# drop neighborhood w/o venues
pd.options.mode.use_inf_as_na = True
toronto_merged.dropna(subset=['Cluster Labels'], inplace=True)

#convert Cluster Labels to integer
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,3,coffee,italian,pizza,burgers,breakfast_brunch,sushi,sandwiches,bakeries,chinese,dog_parks
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188,3,coffee,cafes,sandwiches,salad,hotdogs,sushi,landmarks,desserts,indpak,burgers
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804,3,coffee,hotdogs,japanese,mideastern,pizza,mexican,bubbletea,burgers,cafes,newcanadian
22,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587,3,coffee,sandwiches,breakfast_brunch,cafes,italian,newcanadian,bakeries,hotdogs,pizza,burgers
30,M4E,East Toronto,The Beaches,43.67709,-79.29547,1,diners,parks,sushi,coffee,burgers,pubs,chinese,cafes,divebars,ethiopian


Finally, let's visualize the resulting clusters


In [28]:
# create map
map_clusters = folium.Map(location=[toronto_lat, toronto_lng], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters


Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster. 

In [29]:
# create a new dataframe with only the relevant columns
toronto_clusters = toronto_merged.drop(['PostalCode', 'Latitude', 'Longitude'], axis = 1)

#### Cluster 0


In [30]:
toronto_clusters[toronto_clusters['Cluster Labels'] == 0]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
129,Central Toronto,"Moore Park, Summerhill East",0,parks,zoos,dinnertheater,farmersmarket,falafel,ethiopian,egyptian,dumplings,donuts,dog_parks


#### Cluster 1 


In [31]:
toronto_clusters[toronto_clusters['Cluster Labels'] == 1]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,East Toronto,The Beaches,1,diners,parks,sushi,coffee,burgers,pubs,chinese,cafes,divebars,ethiopian
66,East Toronto,"The Danforth West, Riverdale",1,hotdogs,parks,coffee,venues,cafes,divebars,farmersmarket,falafel,ethiopian,egyptian


#### Cluster 2

In [32]:
toronto_clusters[toronto_clusters['Cluster Labels'] == 2]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
111,Central Toronto,"North Toronto West, Lawrence Park",2,coffee,desserts,zoos,divebars,filipino,farmersmarket,falafel,ethiopian,egyptian,dumplings


#### Cluster 3


In [33]:
toronto_clusters[toronto_clusters['Cluster Labels'] == 3]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Downtown Toronto,"Regent Park, Harbourfront",3,coffee,italian,pizza,burgers,breakfast_brunch,sushi,sandwiches,bakeries,chinese,dog_parks
6,Downtown Toronto,"Queen's Park, Ontario Provincial Government",3,coffee,cafes,sandwiches,salad,hotdogs,sushi,landmarks,desserts,indpak,burgers
13,Downtown Toronto,"Garden District, Ryerson",3,coffee,hotdogs,japanese,mideastern,pizza,mexican,bubbletea,burgers,cafes,newcanadian
22,Downtown Toronto,St. James Town,3,coffee,sandwiches,breakfast_brunch,cafes,italian,newcanadian,bakeries,hotdogs,pizza,burgers
31,Downtown Toronto,Berczy Park,3,coffee,italian,newcanadian,pubs,bakeries,sandwiches,breakfast_brunch,pizza,cafes,japanese
40,Downtown Toronto,Central Bay Street,3,coffee,hotdogs,japanese,sandwiches,landmarks,chinese,thai,desserts,newcanadian,bubbletea
41,Downtown Toronto,Christie,3,korean,coffee,cafes,bubbletea,parks,mexican,bakeries,salvadoran,landmarks,catering
49,Downtown Toronto,"Richmond, Adelaide, King",3,coffee,sandwiches,bakeries,breakfast_brunch,salad,newcanadian,cafes,sushi,hotdogs,italian
50,West Toronto,"Dufferin, Dovercourt Village",3,portuguese,bars,hotdogs,restaurants,laundromat,mediterranean,mexican,salad,bakeries,coffee
58,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",3,coffee,newcanadian,hotdogs,breakfast_brunch,pizza,italian,japanese,sandwiches,salad,sushi


#### Cluster 4

In [34]:
toronto_clusters[toronto_clusters['Cluster Labels'] == 4]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
147,Downtown Toronto,Rosedale,4,bakeries,parks,dog_parks,zoos,farmersmarket,falafel,ethiopian,egyptian,dumplings,donuts
